In [123]:
import pandas as pd

# Load the data from CSV file
df = pd.read_csv('source files/ACSST1Y2021.S2503-2023-04-21T042648.csv')

# Filter the columns containing "Renter-occupied housing units!!Estimate"
renter_columns = [col for col in df.columns if "Renter-occupied housing units!!Estimate" in col]
renter_df = df[['Label (Grouping)'] + renter_columns]

# Remove extra spaces from the 'Label (Grouping)' column and rename it to 'households'
renter_df['Label (Grouping)'] = renter_df['Label (Grouping)'].str.strip()
renter_df = renter_df.rename(columns={'Label (Grouping)': 'households'})

# Filter rows based on the specified text chunks
desired_labels = [
    'Less than $5,000',
    '$5,000 to $9,999',
    '$10,000 to $14,999',
    '$15,000 to $19,999',
    'Occupied housing units',
    '$20,000 to $24,999',
    '$25,000 to $34,999',
    '$50,000 to $74,999',
    '$75,000 to $99,999',
    '$100,000 to $149,999',
    '$150,000 or more'

]

filtered_df = renter_df[renter_df['households'].isin(desired_labels)]

# Convert string values with commas to integers
for col in filtered_df.columns[1:]:
    filtered_df[col] = filtered_df[col].apply(lambda x: int(x.replace(',', '')) if isinstance(x, str) else x)

# Create a DataFrame without the 'Occupied housing units' row for the total calculation
calculation_df = filtered_df[filtered_df['households'] != 'Occupied housing units']

# Calculate the total for each column (except the first one) and create a new row with the "total" label
total_row = pd.DataFrame({'households': 'Total'}, index=[0])
for col in calculation_df.columns[1:]:
    total_row[col] = calculation_df[col].sum()

# Append the total row to the filtered DataFrame
filtered_df = pd.concat([filtered_df, total_row], ignore_index=True)

# Find the index of the "Total" and "Occupied housing units" rows
total_index = filtered_df.index[filtered_df['households'] == 'Total'][0]
occupied_index = filtered_df.index[filtered_df['households'] == 'Occupied housing units'][0]

# Calculate the percentage for each column (except the first one) and create a new row with the "percentage" label
percentage_row = pd.DataFrame({'households': 'Percentage under 24,999'}, index=[0])
for col in filtered_df.columns[1:]:
    percentage = (filtered_df.loc[total_index, col] / filtered_df.loc[occupied_index, col]) * 100
    percentage_row[col] = round(percentage)

# Append the percentage row to the filtered DataFrame
filtered_df = pd.concat([filtered_df, percentage_row], ignore_index=True)

# Strip out the ", New Jersey!!Renter-occupied housing units!!Estimate" from each column header except the first
filtered_df.columns = ['households'] + [col.replace(', New Jersey!!Renter-occupied housing units!!Estimate', '') for col in filtered_df.columns[1:]]
filtered_df.columns = ['households'] + [col.replace('!!Renter-occupied housing units!!Estimate', '') for col in filtered_df.columns[1:]]


# Format the numbers as strings with commas (except for the first column)
formatted_df = filtered_df.copy()
formatted_df.iloc[:, 1:] = formatted_df.iloc[:, 1:].applymap(lambda x: '{:,}'.format(x))


# Strip the word "County", remove spaces, and change the remaining text to uppercase
formatted_df.columns = ['households'] + [col.replace(' County', '').upper() for col in formatted_df.columns[1:]]


# Transpose the DataFrame
transposed_df = formatted_df.T


# Update the index with the desired text
transposed_df.index = transposed_df.index.to_series().replace({'households': 'COUNTY',
                                                               'Total': 'TotalBelow25000',
                                                               'Occupied housing units': 'Occupied rental housing units'})

# Find and replace the desired text in the transposed DataFrame
transposed_df = transposed_df.replace({'households': 'COUNTY',
                                       'Total': 'TotalBelow25000',
                                       'Occupied housing units': 'Occupied rental housing units'})


# Save the resulting DataFrame to a CSV file
transposed_df.to_csv('renter_household_income_filtered.csv', header=False)

# Display the result
display(pd.concat([transposed_df.head(50)]))



C:\Users\BGROSS\AppData\Local\Temp\ipykernel_45400\3806433150.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renter_df['Label (Grouping)'] = renter_df['Label (Grouping)'].str.strip()
C:\Users\BGROSS\AppData\Local\Temp\ipykernel_45400\3806433150.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[col] = filtered_df[col].apply(lambda x: int(x.replace(',', '')) if isinstance(x, str) else x)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
COUNTY,Occupied rental housing units,"Less than $5,000","$5,000 to $9,999","$10,000 to $14,999","$15,000 to $19,999","$20,000 to $24,999","$25,000 to $34,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 or more","$50,000 to $74,999",TotalBelow25000,"Percentage under 24,999"
NEW JERSEY,"1,244,971","66,661","46,056","81,661","66,926","60,494","117,830","216,411","146,478","157,918","135,531","211,793","1,307,759",105
ATLANTIC,"37,101","2,842","1,435","3,289","3,238","3,156","5,426","5,300","3,148","2,836","2,062","4,990","37,722",102
BERGEN,"117,928","6,936","3,683","4,251","4,934","4,158","10,165","23,789","14,736","18,145","14,638","23,108","128,543",109
BURLINGTON,"39,718","2,275","1,309","2,076","1,488","1,817","4,755","6,645","6,087","4,992","3,196","6,383","41,023",103
CAMDEN,"69,773","3,730","4,021","4,836","4,905","3,924","7,218","13,266","6,410","8,302","3,591","12,808","73,011",105
CAPE MAY,"10,697",443,417,"1,072",430,"1,029","1,969","1,712","1,154",804,343,"1,331","10,704",100
CUMBERLAND,"18,617","1,370",832,"2,043",878,"1,389","3,218","3,790","1,699",329,445,"3,754","19,747",106
ESSEX,"178,779","12,895","9,138","13,239","14,079","9,479","16,942","34,325","17,562","16,616","11,790","34,049","190,114",106
GLOUCESTER,"21,823","1,043",565,"2,299","1,767","1,129","2,132","3,879","2,430","1,992","2,115","3,879","23,230",106
